# PREPROCESSING IMAGE DATA TO LINKED DATA
In this notebook, we evaluate the image assets in a session repository.
For every jpeg/png with its accompanying xmp or xmp file, an IMAGENODE metadata class is created 
that governs all the metadata of the image (paths, pose, etc.).
As ouput, the method generates RDF GRAPHS (.ttl) files

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [154]:
from rdflib import Graph, plugin
import uuid    
import importlib
import PIL.Image as PILimage
import os.path
import open3d   as o3d
import cv2
#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

In [155]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [156]:
%autoreload 2

## 1. INITIALIZE SESSION

In [157]:
## INPUTS
projectPath= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"tests")#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = os.path.join(projectPath , "Sample2") #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon
graphPath= os.path.join(sessionPath,"sessionGraph.ttl") 

## 2. READ PRESENT JPEG/PNG/XMP/XML CONTAINING IMAGES + METADATA
An IMAGENODE is created per image in the session

In [158]:
flightnode1=MeshNode(path=os.path.join(sessionPath,'101_0366','MeshLAMBERT72_TAW.ply'),getResource=True) 
flightnode2=MeshNode(path=os.path.join(sessionPath,'101_0367','MeshLAMBERT72_TAW.ply'),getResource=True)


In [159]:
ifcnodelist=tl.ifc_to_nodes(os.path.join(sessionPath,'Mariakerke_AWV_Conform_3D_BT_l72.ifc'),classes='.IfcSite',getResource=True)

In [160]:
print(len(ifcnodelist))
print(ifcnodelist[0].resource)

65
TriangleMesh with 8 points and 12 triangles.


In [161]:
geometrylist=[node.resource.get_oriented_bounding_box() for node in ifcnodelist if node.resource is not None]
meshlist = []
for node in ifcnodelist:
    if node.resource is not None:
        meshlist.append(node.resource)
        # meshlist[-1].paint_uniform_color([0,0,1])
meshlist.append(flightnode1.resource)
meshlist[-1].paint_uniform_color([0,0,1])
meshlist.append(flightnode2.resource)
meshlist[-1].paint_uniform_color([1,0,0])

o3d.visualization.draw_geometries(meshlist)


In [162]:
# path=sessionPath+"\\100_0848_0005.JPG"
# img=cv2.imread(path)
# graphPath=sessionPath+"\\imgGraph.ttl"
# xmpPath=sessionPath+"\\IMG_2173.xmp"
# node1=ImageNode(path=path, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)
# node2=ImageNode(img=img, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)
# # node3=ImageNode(graphPath=graphPath, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)
# # node3=ImageNode(xmpPath=xmpPath, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)


In [163]:
# print(node1.subject)
# print(node1.geospatialTransform)

In [164]:
# path='C:\\RepoHeinder\\geomapi\\tests\\Sample3\\101_0366_0036.jpg'
# subject='101_0366_0036'
# fileformat=['.jpg','.png']
# print(ut.get_extension(path))
# print(subject)
# print(path)
# if ut.get_extension(path) in fileformat:
#     print('wowowo')
# if subject in path:
#     print('jippie')

In [165]:
import xml.etree.ElementTree as ET 
import numpy as np
from rdflib import URIRef
# Retrieve all image files in the session along with their xml metadata
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
nodelist=[]

for path in allSessionFilePaths:        
    
    if path.endswith(".JPG") or path.endswith(".PNG"): 
        testXmpPath=path.replace('JPG','xmp')
        if testXmpPath in allSessionFilePaths:
            nodelist.append(ImageNode(path=path,xmpPath=testXmpPath,sessionPath=sessionPath))
        else:
            nodelist.append(ImageNode(path=path,sessionPath=sessionPath))

    if path.endswith(".xml"):
        nodelist.extend(tl.img_xml_to_nodes(path, getResource=True,graphPath=graphPath))
          
print(str(len(nodelist))+' ImageNodes are created.')
# print(str(exifCounter)+' have poses in exif data')
# print(str(xmpCounter)+' have poses in xmp data')
# print(str(xmlCounter)+' have poses in xml data (not implemented yet)')
print('Note that these nodes do not yet contain actual data but only the metadata')

156 ImageNodes are created.
Note that these nodes do not yet contain actual data but only the metadata


## 3. CREATE RDF GRAPH FOR All IMAGENODES AND EXPORT THEM TO .TTL
An RDF graph and .ttl file is created for all images in the session
(images themselves are not stored in the graph, only metadata)

In [166]:
graph=tl.nodes_to_graph(nodelist,graphPath=graphPath)
subjectList=[1 for node in nodelist if node.graph is not None]
# print (str(len(subjectList))+' Nodes succesfully serialized in: '+ graphPath)
graphifc=tl.nodes_to_graph(ifcnodelist)
subjectListifc=[1 for node in ifcnodelist if node.graph is not None]
# graphifc.serialize(destination=graphPath)
# print(str(len(subjectList)))
# print(str(len(subjectList)))
# print(str(len(subjectListifc)))
subjectList = subjectList + subjectListifc
graph=graph+graphifc
graph.serialize(destination=graphPath)
print (str(len(subjectList))+' Nodes succesfully serialized in: '+ graphPath)

221 Nodes succesfully serialized in: c:\RepoHeinder\geomapi\tests\Sample2\sessionGraph.ttl
